# Anime NeuralMF Hybrid Recommender
### In this notebook, we implement a recommender model with the MyAnimeList Anime Recommendations dataset.

> Based on the Neural Collaborative Filtering paper: Xiangnan He, Lizi Liao, Hanwang Zhang, Liqiang Nie, Xia Hu and Tat-Seng Chua (2017). Neural Collaborative Filtering. In Proceedings of WWW '17, Perth, Australia, April 03-07, 2017.

#### The following is a little motivation for Hybrid recommender systems.

## Why Hybrid?
Well, there are two main kinds of recommender systems: Content-based and Collaborative filtering-based.
* Content-based recommenders suggest similar picks to a certain _item_ (an anime movie/series in our case), letting the users know about similar items to the ones they have watched/rated positively. These method typically use _item features_ together with unsupervised methods in an effort to generate a product-space and compute similarities between items. However, this method may end suggesting a limited mix of items, providing a low _surprise factor_ for the user.
* On the other hand, collaborative filtering recommenders rely on past users' history of watched/rated items, increasing the chances of recommending a serendipitous item to a target user. Classic methods rely solely on a user-item matrix, which maps the interactions that all users have with every item. These matrix methods are heavily memory-intensive and newer neural network-based are more common. Nonetheless, these methods could miss on similar -but typically overseen- items, in comparison to the ones watched/reviewed by the target user.

In order to get more robust recommendations, a hybrid model can combine both item features and user-item features.

## And... why NeuralMF?
The NeuralMF is a mix of General Matrix Factorization (GMF) and Multi Layer Perceptron (MLP) recommenders, resembling a Wide&Deep model, having higih generalization power. Plus, neural nets make easier to handle large volumes of data, and it better leverages the power of GPUs! For more info, refer to the [article](https://arxiv.org/abs/1708.05031).

In [ ]:
# Some typical imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import OneHotEncoder, QuantileTransformer
from sklearn.model_selection import train_test_split
from numba import jit # Compile some functions when performance is critical
import keras
from keras.initializers import RandomNormal
from keras.models import Model, load_model, save_model
from keras.layers import Embedding, Input, Dense, Concatenate, Multiply, Flatten
from keras.optimizers import Adam
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU")

## Content-based feature engineering

In [ ]:
# Check our data structure
anime = pd.read_csv("../input/anime-recommendations-database/anime.csv")
anime.head(10)

For simplicity, let's use TV anime only. This prevents future complications with the episode number.

In [ ]:
anime = anime[anime['type'] == 'TV']

The content-based part of our model requires features for each anime. Therefore, we are going to include as many relevant features as possible, to not waste any information. So, we are using the anime genres and number of episodes. We dropped the title here, due to a lack of ways of handling it. Rating and members are not content-related features, since they are dynamic and bounded to users' activity. So, they are going to be leveraged through the collaborative-filtering part.

Let's start by one-hot encoding genres:

In [ ]:
# Copy the column
anime['features_genre'] = anime['genre']

# Cast None to an empty string
anime['features_genre'] = anime['features_genre'].fillna('') 
# Split genres into a list of strings
anime['features_genre'] = anime['features_genre'].map(lambda x: x.split(', '))

# Create a set of all genres
all_genres = set()
for row in anime['features_genre']:
    # Union of sets is declared with the | operator
    all_genres = all_genres | set(row)
all_genres.remove('') # Drop the empty genre

def invert_dict(d):
    return {value: key for key, value in d.items()}

all_genres = sorted(list(all_genres)) # We convert it to a list to enforce alphabetic ordering
ngenres = len(all_genres)

idx2genre = dict(enumerate(all_genres)) # Create a mapping dictionary from index to dict
genre2idx = invert_dict(idx2genre) # Inverse dict

genre2idx

In [ ]:
def encode_genres(genres):
    out = np.zeros(ngenres)
    for genre in genres:
        if genre == '':
            pass
        else:
            out[genre2idx[genre]] = 1
    return out.tolist()
anime['features_genre'] = anime['features_genre'].map(encode_genres)
anime['features_genre'] # See how the encoded features look

Now, we need to do some more detailed feature engineering for the remaining features.

The number of episodes contains 'Unknown' labels among numeric values. We impute them by filling missed episodes with 1 episode.

In [ ]:
anime['features_episodes'] = anime['episodes'].replace({'Unknown' : 1}).astype(np.int32)
sb.distplot(anime['features_episodes']);
# This feature is heavily unbalanced! Let's apply a quantile transformation to it

Here, we impose a uniform-like distribution, using Scikit-Learn's QuantileTransformer. This is an easier-to-handle representation.

In [ ]:
ep_discretizer = QuantileTransformer(n_quantiles = 100)
feats_ep = anime['features_episodes'].apply(np.log).to_numpy().reshape(-1, 1)
feats_ep = ep_discretizer.fit_transform(feats_ep).flatten().tolist()
anime['features_episodes'] = feats_ep
sb.distplot(anime['features_episodes']);

## Collaborative-filtering feature engineering

In [ ]:
# Check our data structure
rating = pd.read_csv("../input/anime-recommendations-database/rating.csv")
rating.head(10)

## We lower the number of users for quicker exploration & training, although you can comment the next line.
### Everything fits in memory, so it won't crash!

In [ ]:
rating = rating[rating['user_id'] <= 10000] # Can comment this line
rating = rating[rating['anime_id'].isin(anime['anime_id'])] # Don't comment this one though!

## Our goal is to reach as many users as possible; we need to impute missing reviews

Many users don't review the shows they've watched. We could drop these records or impute them with the median of the users' ratings, for example.
Inspecting the distribution of ratings, we see that most of them are positive (with a median of 8). Therefore, we should consider imputing unrated shows, since most seen shows are positive signals.

An important thing to stress is that our algorithm is not trying to recommend masterpieces only, but rather a varied mix of shows that the user might enjoy more or less. This helps to reach _hardcore otakus_ as well as casual viewers.

In [ ]:
print(rating['rating'].replace({-1: np.nan}).dropna().describe())
sb.distplot(rating['rating'], kde = False);

We group by user and replace missing ratings with the median of the user.
Many users don't leave reviews. To not lose this information, we impute them with the median of all users: 8

In [ ]:
user_median = rating.groupby('user_id').median()['rating']
sb.distplot(user_median, kde = False);
overall_median = user_median.median()
print("Median of all users' medians: ", overall_median)
user_median = dict(user_median.replace({-1 : overall_median}))

In [ ]:
user_medians = rating['user_id'].apply(lambda x: user_median[x])
rating['rating'] = rating['rating'].replace({-1 : np.nan}).fillna(user_medians)
rating['rating'] = rating['rating'] / rating['rating'].max() # Divide by the max to normalize!

In [ ]:
# Resulting histogram
sb.distplot(rating['rating'], kde = False);

## Construct training and testing sets
Our current dataset is incomplete, since we need to generate rows including anime that users' havent watched (_negative intances_). The following accounts for that factor. We need to emphasize that we don't want every user to have a row for every anime, to not fill up our entire RAM memory.

Allow us to set that every rating will trigger 4 negative entries (we picked 4 just as a fiducial value from the original repo). To generate these records, we simply sample 4 unwatched animes for each user rating.

In [ ]:
num_neg = 4
user2n_anime = dict(rating.groupby('user_id').count()['anime_id'])

In [ ]:
all_users = np.sort(rating['user_id'].unique())
all_anime = np.sort(rating['anime_id'].unique())
n_anime = len(all_anime)
n_users = len(all_users)

@jit
def choice_w_exclusions(array, exclude, samples):
    max_samples = len(array)-len(exclude)
    final_samples = min(samples, max_samples)
    possible = np.array(list(set(array) - set(exclude)))
    return np.random.choice(possible, size = final_samples, replace = False)
@jit
def flat(l):
    return [item for sublist in l for item in sublist]

### Sample negative entries

In [ ]:
%%time
#This part takes about 10 minutes with a full dataset. Time for coffee!
neg_user_id = []
neg_anime_id = []
neg_rating = []

for user in all_users:
    exclude = list(rating[rating['user_id'] == user]['anime_id'])
    sampled_anime_id = choice_w_exclusions(all_anime, exclude, len(exclude) * num_neg)
    
    neg_user_id.append([user] * len(sampled_anime_id))
    neg_anime_id.append(sampled_anime_id)
    neg_rating.append([0.] * len(sampled_anime_id))
    
neg_user_id = flat(neg_user_id)
neg_anime_id = flat(neg_anime_id)
neg_rating = flat(neg_rating)

In [ ]:
negatives = pd.DataFrame({'user_id': neg_user_id,
                          'anime_id': neg_anime_id,
                          'rating': neg_rating})
data = pd.concat([rating, negatives], ignore_index = True)

### Join both tables' information and drop unindexed anime

In [ ]:
anime['features'] = anime['features_genre'] + anime['features_episodes'].apply(lambda x: [x])
anime['features'] = anime['features'].apply(np.array)
n_feats = len(anime['features'].iloc[0])
data = data.join(anime['features'], on = 'anime_id').dropna()

Embeddings need a compressed index representation of animes: Let's make a quick mapping

In [ ]:
anime2item_dict = dict(zip(np.sort(all_anime), list(range(n_anime))))
item2anime_dict = {v: k for k, v in anime2item_dict.items()}

def anime2item(a_id):
    return anime2item_dict[a_id]

def item2anime(i_id):
    return item2anime_dict[i_id]
                       
data['item_id'] = data['anime_id'].apply(anime2item)

### Split into a 90/10 train/test scheme.
Note: We can't separate users between train and test sets (like train users versus test users), since we need to feed all users and anime shows to the embeddings.

In [ ]:
x0 = data['user_id'].to_numpy()
x1 =data['item_id'].to_numpy()
x2 = np.stack(data['features'].to_numpy())
y = data['rating'].to_numpy()

(x0_train, x0_val,
 x1_train, x1_val,
 x2_train, x2_val,
 y_train, y_val) = train_test_split(x0, x1, x2, y,
                                    test_size = 0.1,
                                    random_state = 42)

x_train = [x0_train, x1_train, x2_train]
x_val = [x0_val, x1_val, x2_val]

## Model implementation
> [Heavily based on the [Neural Collaborative Filtering paper repo](https://github.com/hexiangnan/neural_collaborative_filtering)]

However, our model improved the reference model by including information of anime features!

In [ ]:
def get_model(num_users, num_items, num_item_feats, mf_dim, layers = [64, 32, 16, 8]):
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    feats_input = Input(shape=(num_item_feats,), dtype='float32', name = 'feats_input')

    # User&Item Embeddings for Matrix Factorization
    MF_Embedding_User = Embedding(input_dim = num_users + 1, output_dim = mf_dim,
                                  name = 'user_embedding',
                                  embeddings_initializer = RandomNormal(stddev=0.001),
                                  input_length = 1)
    MF_Embedding_Item = Embedding(input_dim = num_items + 1, output_dim = mf_dim,
                                  name = 'item_embedding',
                                  embeddings_initializer = RandomNormal(stddev=0.001),
                                  input_length = 1)
    
    # User&Item Embeddings for MLP part
    MLP_Embedding_User = Embedding(input_dim = num_users + 1, output_dim = int(layers[0] / 2),
                                   name = 'mlp_embedding_user',
                                   embeddings_initializer = RandomNormal(stddev=0.001),
                                   input_length = 1)
    MLP_Embedding_Item = Embedding(input_dim = num_items + 1, output_dim = int(layers[0] / 2),
                                   name = 'mlp_embedding_item',
                                   embeddings_initializer = RandomNormal(stddev=0.001),
                                   input_length = 1) 
    
    mf_user_latent = Flatten()(MF_Embedding_User(user_input))
    mf_item_latent = Flatten()(MF_Embedding_Item(item_input))
    mf_vector = Multiply()([mf_user_latent, mf_item_latent])

    # MLP part with item features
    mlp_user_latent = Flatten()(MLP_Embedding_User(user_input))
    mlp_item_latent = Flatten()(MLP_Embedding_Item(item_input))
    
    mlp_vector = Concatenate()([mlp_user_latent, mlp_item_latent, feats_input])
    for l in layers:
        layer = Dense(l, activation='relu')
        mlp_vector = layer(mlp_vector)

    # Concatenate MF and MLP parts
    predict_vector = Concatenate()([mf_vector, mlp_vector])
    
    # Final prediction layer
    prediction = Dense(1, activation = 'sigmoid',
                       kernel_initializer = 'lecun_uniform',
                       name = 'prediction')(predict_vector)
    
    model = Model(input = [user_input, item_input, feats_input], output = prediction)
    return model

Set hyperparameters, which are very similar to the default values from the NeuralMF model repo, except for the number of epochs and layers.

In [ ]:
learning_rate = 0.001
batch_size = 256
n_epochs = 3
mf_dim = 15
layers = [128, 64, 32, 16, 8]

Create model and train!

In [ ]:
model = get_model(n_users, n_anime, n_feats, mf_dim, layers)
model.compile(optimizer = Adam(lr = learning_rate), loss = 'mean_squared_logarithmic_error')

In [ ]:
hist = model.fit(x = x_train, y = y_train, validation_data = (x_val, y_val),
                 batch_size = batch_size, epochs = n_epochs, verbose = True, shuffle = True)

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Eval'], loc = 'upper right')
plt.show()

### Alright, let's visualize some recommendations!

In [ ]:
indexed_anime = anime.set_index('anime_id')

def explore(user_id, top = 5):
    sub = rating[rating['user_id'] == user_id]
    watched_animes = sub['anime_id']
    ratings = sub['rating']
    names = indexed_anime.loc[watched_animes]['name']
    genres = indexed_anime.loc[watched_animes]['genre']
    rating_info = pd.DataFrame(zip(watched_animes, names,
                                   genres, ratings * 10),
                               columns = ['anime_id', 'name',
                                          'genre', 'rating']).set_index('anime_id')
    return rating_info.sort_values(by = 'rating', ascending = False).iloc[:top]

def recommend(user_id, recommendations = 5):
    watched_animes = rating[rating['user_id'] == user_id]['anime_id']
    
    test_anime = np.array(list(set(all_anime) - set(watched_animes)))
    test_user = np.array([user_id] * len(test_anime))
    test_items = np.array([anime2item(a) for a in test_anime])
    sub_anime = indexed_anime.loc[test_anime]
    test_features = np.stack(sub_anime['features'].to_numpy())
    test = [test_user, test_items, test_features]
    preds = model.predict(test).flatten()
    results = pd.DataFrame(zip(sub_anime['name'], test_anime,  sub_anime['genre'], preds * 10),
                           columns = ['name', 'anime_id',
                                      'genre', 'score']).set_index('anime_id')
    return results.sort_values(by = 'score', ascending = False).iloc[:recommendations]

In [ ]:
explore(444) # Action Sports study

In [ ]:
recommend(444)

In [ ]:
explore(999) # Action Fantasy case study

In [ ]:
recommend(999)

In [ ]:
explore(111) # Techno study

In [ ]:
recommend(111)

## Apparently, the recommender works like a charm.
But... There's still plenty of job to do, like observing extreme cases such as users with few watched anime, niche clusters, and so on.